In [6]:
pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.9/815.9 kB 12.3 MB/s eta 0:00:00


In [7]:
pip install langchain-community

In [8]:
pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 4.5 MB/s eta 0:00:00


In [9]:
pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 20.3 MB/s eta 0:00:00


In [1]:
from google.colab import drive

drive.mount("gdrive")

Mounted at gdrive


In [ ]:
!cp gdrive/MyDrive/kdt_jyg/workspace/Dacon_hansol/*.csv .

In [ ]:
import pandas as pd
train = pd.read_csv("/content/gdrive/MyDrive/kdt_jyg/workspace/Dacon_hansol/train0214.csv", encoding='cp949')

from itertools import product

# 빈 리스트 생성
train_data = []

# 질문과 답변의 조합을 생성
for q, a in product([f"질문_{x}" for x in range(1, 3)], [f"답변_{x}" for x in range(1, 6)]):
    for i in range(len(train)):
        # 값이 문자열이 아닌 경우를 고려하여 str() 함수를 사용
        question = str(train.at[i, q])
        answer = str(train.at[i, a])
        train_data.append("질문: " + question + " 답변 : " + answer)

In [ ]:
len(train_data)

7640

In [ ]:
pd.DataFrame(train_data).to_csv("train_data.csv",index=False,encoding='cp949')

In [ ]:
# 단순화 시작부(csv 단순화 파일 삽입.)

In [6]:
from langchain_community.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path='/content/gdrive/MyDrive/kdt_jyg/workspace/Dacon_hansol/train_data2155.csv',encoding='utf-8')
data = loader.load()

In [7]:
# 커스텀 확인
second_row = data[1]
print(second_row)

page_content='0: 내진설계에는 내진구조, 제진구조, 면진구조가 있습니다. 내진구조는 건물 구조물이 지진의 힘에 따라 변형되는 것을 방지하기 위해 설계됩니다. 제진구조는 건물의 지반을 고정시키거나 지진의 힘을 줄이기 위해 설계되며, 면진구조는 건물을 지진에 노출되는 방향으로부터 보호합니다.\n: ' metadata={'source': '/content/gdrive/MyDrive/kdt_jyg/workspace/Dacon_hansol/train_data2155.csv', 'row': 1}


In [8]:
from langchain_community.embeddings import HuggingFaceEmbeddings

modelPath = "distiluse-base-multilingual-cased-v1"
model_kwargs = {'device':'cuda'}
encode_kwargs = {'normalize_embeddings': False}

embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.47k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

In [15]:
from transformers import Trainer, TrainingArguments
from langchain_community.document_loaders.csv_loader import CSVLoader
from torch.utils.data import Dataset
import pandas as pd

from transformers import LlamaForCausalLM, AutoTokenizer, pipeline
model_id = "beomi/llama-2-ko-7b"
tokenizer = AutoTokenizer.from_pretrained(model_id)

class TextDataset(Dataset):
    def __init__(self, texts, tokenizer):
        self.texts = texts
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        encoding = self.tokenizer.encode_plus(text, return_tensors='pt')
        return encoding

# 데이터 준비
loader = CSVLoader(file_path='/content/gdrive/MyDrive/kdt_jyg/workspace/Dacon_hansol/train_data2155.csv',encoding='utf-8')
data = loader.load()

# A열의 텍스트만 사용
texts = [doc.page_content for doc in data]

# 텍스트를 토큰화하여 데이터셋 생성
train_dataset = TextDataset(texts, tokenizer)

# # 모델 학습
# training_args = TrainingArguments(
#     output_dir="./results",          # 학습 결과를 저장할 디렉토리
#     num_train_epochs=3,              # 학습 에포크 수
#     per_device_train_batch_size=16,  # 배치 크기
#     weight_decay=0.01,               # 가중치 감소
# )

# trainer = Trainer(
#     model=model,                         # 학습시킬 모델
#     args=training_args,                  # 학습 인자
#     train_dataset=train_dataset,         # 학습 데이터셋
# )

# trainer.train()

# # 모델 저장
# model.save_pretrained("./saved_model")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/842 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.55M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [19]:
# 항목 가져오기
first_item = train_dataset[700]

# 토큰화된 텍스트 출력
print("Tokenized text: ", first_item['input_ids'])

# 원래 텍스트로 디코딩
decoded_text = tokenizer.decode(first_item['input_ids'].squeeze().tolist())
print("Decoded text: ", decoded_text)

Tokenized text:  tensor([[    1, 29871, 29900, 29901, 32106, 31417, 45025, 34958, 43876, 33330,
         32130, 35854, 32658, 31906, 35145, 29901,  6756,    13, 30004,    13,
         31940, 31231, 31126, 29901, 32106, 31417, 45025, 34958, 35930, 40865,
         32058, 31231, 32485, 33347, 32701, 44854, 29892, 32130, 34107, 32332,
         32651, 32359, 37937, 32171, 32043, 29889, 39330, 33246, 32017, 32956,
         33449, 32791, 34550, 32196, 38665, 32214, 29889,  6756,    13, 30004,
            13, 31945, 30970, 31126, 29901, 32106, 31417, 45025, 34958, 35930,
         41915, 40072, 32175, 32931, 32094, 30970, 32485, 34600, 32111, 29889,
         39330, 41438, 32144, 32473, 44851, 31906, 32896, 37138, 32472, 32504,
         34221, 32306, 40594, 32337, 29892, 32184, 31054, 38660, 32282, 31158,
         32144, 32359, 38100, 37792, 32111, 29889,  6756,    13, 30004,    13,
         30918, 44977, 36362, 33617, 29901, 32106, 31417, 45025, 34958, 35930,
         34966, 34746, 31327, 35316

In [9]:
from langchain.vectorstores import FAISS

In [10]:
db = FAISS.from_documents(data, embedding=embeddings)

In [11]:
db.save_local("faiss_index")

In [12]:
retriever = db.as_retriever(search_kwargs={"k": 4})

In [13]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import LlamaForCausalLM, AutoTokenizer, pipeline

model_id = "beomi/llama-2-ko-7b"

tokenizer = AutoTokenizer.from_pretrained(model_id)

model = LlamaForCausalLM.from_pretrained(model_id).to(0)

# 메모리를 줄이기 위해 128으로 조정
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=128, device=0)

hf = HuggingFacePipeline(pipeline=pipe)

tokenizer_config.json:   0%|          | 0.00/842 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.55M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/606 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

...

model-00014-of-00015.safetensors:   0%|          | 0.00/742M [00:00<?, ?B/s]

model-00015-of-00015.safetensors:   0%|          | 0.00/380M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 172.00 MiB. GPU 0 has a total capacty of 15.77 GiB of which 142.38 MiB is free. Process 21870 has 15.63 GiB memory in use. Of the allocated memory 15.25 GiB is allocated by PyTorch, and 21.06 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

template = """다음과 같은 맥락에서:
{context}

고객님께서 문의하신 내용에 대한 답변은 다음과 같습니다:
"""

custom_rag_prompt = PromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever}
    | custom_rag_prompt
    | hf
    | StrOutputParser()
)

In [ ]:
for chunk in rag_chain.stream("도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?"):
    print(chunk, end="", flush=True)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 통나무구조 방식의 건물은 통나무를 쌓아 올려 지붕을 만들고 벽체를 만드는 방식입니다. 통나무구조 방식의 건물은 통나무를 쌓아 올려 지붕을 만들고 벽체를 만드는 방식입니다. 통나무구조 방식의 건물은 통나무를 쌓아 올려 지붕을 만들고 벽체를 만드는 방식입니다.​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​

In [ ]:
import pandas as pd

In [ ]:
test = pd.read_csv("test.csv")

In [ ]:
test

,id,질문
0,TEST_000,"방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용..."
1,TEST_001,도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
2,TEST_002,"큐블럭의 단점을 알려주세요. 또한, 압출법 단열판을 사용하는 것의 장점은 무엇인가요?"
3,TEST_003,"철골구조를 사용하는 고층 건물에서, 단열 효과를 높이기 위한 시공 방법은 무엇이 있..."
4,TEST_004,도배지의 완전한 건조를 위해 몇 주 동안 기다려야 하나요?
...,...,...
125,TEST_125,분말 소화기를 사용할 때 주의해야 할 사항은 무엇인가요? 그리고 아파트 도배 평수를...
126,TEST_126,"압출법 보온판의 가장 큰 장점은 무엇인가요?"""
127,TEST_127,평지붕의 누수 문제를 방지하기 위해 수성 벽체용 탄성 방수 도료를 사용하는 것이 어...
128,TEST_128,석고수정이 발생하는 가장 큰 원인은 무엇인가요? 그리고 이를 해결하는 방법에 대해 ...


In [ ]:
from tqdm import tqdm

In [ ]:
result = []



for i in tqdm(range(len(test))):
  _id = test.at[i,'id']
  _q = test.at[i,'질문']
  _a = []
  for chunk in rag_chain.stream(_q):
      _a.append(chunk)
      print(chunk, end="", flush=True)
  result.append(
      {
          "id":_id,
          "대답":" ".join(_a)
      }
  )
  print()

  0%|          | 0/130 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 방청 페인트의 종류에는 다양한 종류가 있습니다. 그 종류에는 광명단 페인트, 방청산화철 페인트, 알루미늄 페인트, 역청질 페인트, 워시 프라이머, 크롬산아연 페인트, 규산염 페인트 등이 있습니다. 이러한 다양한 종류의 방청 페인트가 각자의 특성과 용도에 맞게 사용됩니다.​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​

  1%|          | 1/130 [00:17<38:06, 17.73s/it]

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 도배지에 녹이 묻어나오는 현상은 녹 오염으로 도배지에 붉은색의 녹이 베어나오는 것을 말합니다. 이 현상은 보통 높은 습도나 누수로 인해 발생할 수 있습니다. ​도배지에 녹이 묻어나오는 원인은 다음과 같습니다. ​1. 습도 문제 원인: 높은 습도로 인해 도배지 안쪽의 금속이 산화되어 녹이 도배지에 발생할 수 있습니다. 책임 소재: 건물의 소유자나 거주자가 습기 관리의 책임이 있습니다. 해결 방법: 제습기를 가동하거나 실내의 적정 습도를 유지하기 위해 환기를 통해 습기를 제거하는 것이 중요합니다. 전문가의 도움을 받아 보수 작업을 권장합니다. ​2. 누수 원인: 누수로 도배지가 젖어 있는 상태가 계속되면 곰팡이와 녹이 발생할 수 있습니다. 책임 소재: 누수 문제는 건물의 소유자나 거주자의 관리 책임이 있습니다. 해결 방법: 누수를 해결하고 곰팡이와 녹이 발생한 부분 및 전체 도배지를 교체하는 보수 작업이 필요합니다. 이러한 작업은 전문가의 도움을 받는 것이 좋습니다.​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​

...

 석고수정이 발생하는 가장 큰 원인은 높은 습도와 차가운 표면온도입니다. 물방울이 형성되고 이어서 발생하는 물기는 건물의 손상과 곰팡이 등을 유발할 수 있습니다. 그러므로 건물 내의 습도 및 온도를 관리하고 외부와 내부의 온도차를 줄이는 것이 중요합니다.​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​

 99%|█████████▉| 129/130 [38:18<00:17, 17.97s/it]

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 카페트의 기대 수명은 제품의 품질과 사용 환경에 따라 다를 수 있지만, 일반적으로 6년 정도를 기대할 수 있습니다. 그러나 주로 다다미하거나 가구를 자주 옮기는 공간에서 사용하는 경우에는 수명이 단축될 수 있으니 이 점을 유의하셔야 합니다. 또한 정기적인 청소와 유지보수를 통해 카페트의 수명을 연장할 수 있습니다.​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​

100%|██████████| 130/130 [38:35<00:00, 17.81s/it]

In [ ]:
import pickle

with open("result.pkl",'wb') as f:
  pickle.dump(result,f)

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
_model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

In [ ]:
for i in range(len(result)):
  result[i]['embedding'] = _model.encode(result[i]['대답'].replace("\u200b"," "))

In [ ]:
submission = []

for i in range(len(result)):
  tmp = {"id":result[i]['id'],}
  for j in range(len(result[i]['embedding'])):
    tmp[f'vec_{j}'] = result[i]['embedding'][j]
  submission.append(
      tmp
  )

In [ ]:
pd.DataFrame(submission).to_csv("submission_RAG.csv",index=False)